# 深度学习开发基本流程

## 数据准备

### 数据加载

这里使用CIFAR10数据库，其中包含：

1. 10个可能的对象（类别）
2. 50000个训练图像
3. 10000个测试图像

In [ ]:
from torchvision.datasets import CIFAR10

train_data = CIFAR10(
    root='./train/',
    train=True,
    download=True
)

print(train_data)
print(len(train_data))
print(train_data.data.shape)
print(train_data.targets)
print(train_data.classes)
print(train_data.class_to_idx)

In [2]:
data, label = train_data[0]
print(data)

<PIL.Image.Image image mode=RGB size=32x32 at 0x1847E130DF0>


In [3]:
test_data = CIFAR10(
    root='./test/',
    train=False,
    download=True
)

print(test_data)
print(len(test_data))
print(test_data.data.shape)

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./test/
    Split: Test
10000
(10000, 32, 32, 3)


### 数据变换

数据值需要进行归一化来帮助训练。

需要增强数据来创建更大的数据集，或者从一种类型的对象转换成一个张量。

In [4]:
from torchvision import transforms

# 定义变换序列
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2023, 0.1994, 0.2010)
    )
])

train_data = CIFAR10(
    root='./train/',
    train=True,
    download=True,
    transform=train_transforms
)

print(train_data)

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./train/
    Split: Train
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )


In [5]:
# 定义变换序列
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.4914, 0.4822, 0.4465),
        std=(0.2023, 0.1994, 0.2010)
    )
])

test_data = CIFAR10(
    root='./test/',
    train=False,
    download=True,
    transform=train_transforms
)

print(test_data)

Files already downloaded and verified
Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./test/
    Split: Test
    StandardTransform
Transform: Compose(
               RandomCrop(size=(32, 32), padding=4)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.201))
           )


### 数据批处理

In [6]:
from torch.utils.data import DataLoader

trainloader = DataLoader(
    train_data,
    batch_size=16,
    shuffle=True
)

testloader = DataLoader(
    test_data,
    batch_size=16,
    shuffle=False
)

## 模型开发

### 模型设计

这里使用的是现代版本的LeNet5模型

In [7]:
import torch
from torch import nn
import torch.nn.functional as F


class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


device = ('cuda' if torch.cuda.is_available() else 'cpu')
model = LeNet5().to(device=device)

In [8]:
from torch import optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(
    model.parameters(),
    lr=0.001,
    momentum=0.9
)

#### 基本训练循环与验证

In [9]:
from torch.utils.data import random_split

train_set, val_set = random_split(
    train_data,
    [40000, 10000]
)

trainloader = DataLoader(
    train_set,
    batch_size=16,
    shuffle=True
)

valloader = DataLoader(
    val_set,
    batch_size=16,
    shuffle=True
)

In [10]:
N_EPOCHS = 10
for epoch in range(N_EPOCHS):

    # train
    train_loss = 0.0
    model.train()
    for inputs, labels in trainloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    # validation
    val_loss = 0.0
    model.eval()
    for inputs, labels in valloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item()

    print("Epoch: {} Train Loss: {} Val Loss: {}"
        .format(
        epoch,
        train_loss / len(trainloader),
        val_loss / len(valloader)
    ))

Epoch: 0 Train Loss: 2.3020647410392763 Val Loss: 2.301087252044678
Epoch: 1 Train Loss: 2.2936175994873045 Val Loss: 2.2653114768981935
Epoch: 2 Train Loss: 2.2162071242332457 Val Loss: 2.184188475227356
Epoch: 3 Train Loss: 2.17371591129303 Val Loss: 2.157452758407593
Epoch: 4 Train Loss: 2.144780662345886 Val Loss: 2.144109496498108
Epoch: 5 Train Loss: 2.120556972408295 Val Loss: 2.1246980798721316
Epoch: 6 Train Loss: 2.1058483763694764 Val Loss: 2.096804043197632
Epoch: 7 Train Loss: 2.090536642408371 Val Loss: 2.0874464437484743
Epoch: 8 Train Loss: 2.0780237338542937 Val Loss: 2.0834509716033938
Epoch: 9 Train Loss: 2.063861628007889 Val Loss: 2.0695509983062745


### 测试

In [11]:
num_correct = 0.0
for x_test_batch, y_test_batch in testloader:
    model.eval()
    y_test_batch = y_test_batch.to(device)
    x_test_batch = x_test_batch.to(device)
    y_pred_batch = model(x_test_batch)
    _, predicted = torch.max(y_pred_batch, 1)
    num_correct += (predicted == y_test_batch).float().sum()
    # print(x_test_batch)
    # print(predicted)
    # print(y_pred_batch)

accuracy = num_correct / (len(testloader) * testloader.batch_size)
print(len(testloader), testloader.batch_size)
print("Test Accuracy: {}".format(accuracy))

625 16
Test Accuracy: 0.3928000032901764


## 模型部署

### 保存模型

In [12]:
torch.save(model.state_dict(), './lenet5_model.pt')
model = LeNet5().to(device)
model.load_state_dict(torch.load('./lenet5_model.pt'))

<All keys matched successfully>